### Import libraries 

In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, PrintfTickFormatter, Title, Text

output_notebook()

Loading BokehJS ...

### Our objetive 

<img src="../../images/figure_3.png">

#### Preprocessing 

In [2]:
dataset = pd.read_csv('../../data/data_figure_3.csv')
dataset.shape

(5, 5)

In [3]:
dataset.head()

,interest,before,after,formatted_before,formatted_after
0,Bored,11,12,11%,12%
1,Not great,5,6,5%,6%
2,OK,40,14,40%,14%
3,Kind of interested,25,30,25%,30%
4,Excited,19,38,19%,38%


In [4]:
dataset_hbar = dataset[['interest', 'before', 'after']].set_index('interest').T.rename_axis('When').sort_values('When', ascending=True).reset_index()

In [5]:
dataset_hbar.head()

interest,When,Bored,Not great,OK,Kind of interested,Excited
0,after,12,6,14,30,38
1,before,11,5,40,25,19


In [6]:
dataset_hbar.columns.name = None
dataset_hbar['When'] = dataset_hbar['When'].apply(lambda x: x.upper())

In [7]:
dataset_hbar

,When,Bored,Not great,OK,Kind of interested,Excited
0,AFTER,12,6,14,30,38
1,BEFORE,11,5,40,25,19


In [8]:
# Bokeh doesn't allow you to add off-axis text. This is a little trick to do it
data = []

new_row = {'When':' ', 'Bored':np.nan, 'Not great':np.nan, 'OK':np.nan, 'Kind of interested':np.nan, 'Excited':np.nan}
data.insert(0, new_row)

In [9]:
data = pd.DataFrame(data)[dataset_hbar.columns]

In [10]:
dataset_hbar = pd.concat([pd.DataFrame(data), dataset_hbar], ignore_index=True, axis=0)

#### Colors 

In [11]:
GRAY_1, GRAY_2, GRAY_3 = '#A6A6A5', '#929497', '#231F20'
GRAY_4, GRAY_5, GRAY_6 = '#838383', '#555655', '#828282'
GRAY_7, GRAY_8, GRAY_9 = '#646369', '#333333', '#9c9c9c'
GRAY_10 = '#76787B'
BLUE_1, BLUE_2, BLUE_3 = '#94B2D7', '#4A81BF', '#174A7E'
ORANGE_1 = '#F79747'

#### Font 

In [12]:
FONT = 'Arial'

#### Plot 

In [13]:
# Set the source of the plot
source = ColumnDataSource(dataset_hbar)

interested = dataset['interest'].to_list()
when = dataset_hbar['When'].to_list()


# Set color by category
colors = [GRAY_4, GRAY_4, ORANGE_1, BLUE_2, BLUE_2]


# Create the figure
p = figure(x_range=(-15, 100),
           x_axis_location='above',
           y_range=when,
           plot_height=450,
           plot_width=650, 
           title='Pilot program was a success' + ' ' * 45,
           toolbar_location='below')


# Add bars to the figure
p.hbar_stack(interested, 
             y='When', 
             height=0.7, 
             color=colors,
             line_width=1.2, 
             line_color='white',
             source=source)


# Add top legend
p.add_layout(Title(text=' ' * 15 + "% total", 
                   align="left", 
                   text_font_size='13pt', 
                   text_color=GRAY_4, 
                   text_font_style = 'normal'), 
             "above")

p.add_layout(Title(text="Bored     |      No great      |      Ok      |      Kind of interested      |    Excited", 
                   align="right", 
                   text_font_size='13pt', 
                   text_color=GRAY_4, 
                   text_font_style='normal'), 
             "above")


# Add subtitles
p.add_layout(Title(text='How do you feel about science?', 
                   align="left", 
                   text_font_size='14pt', 
                   text_color=GRAY_3, 
                   text_font_style='normal'), 
             "above")


# Add footing legend
p.add_layout(Title(text='Based on survey of 100 students conducted before and after pilot program (100% response rate on both surveys).', 
                   align="left", 
                   text_font_size='8.8pt', 
                   text_color=GRAY_9), 
             "below")


# Add text annotations
# This is a weakness of bokeh, you cannot format a single word within a text. We have to do it separately.
p.text(x=0, 
       y=0, 
       y_offset=-58,
       text=['BEFORE program, the'],
       text_font_size='13pt', 
       text_color=GRAY_4)

p.text(x=0, 
       y=0,
       y_offset=-10,
       text=['majority of children (40%) \nfelt just Ok'],
       text_font_size='13pt', 
       text_color=ORANGE_1)

p.text(x=0, 
       y=0,
       y_offset=-10,
       text=['\t' * 18 + 'about science.'],
       text_font_size='13pt', 
       text_color=GRAY_4)

p.text(x=55, 
       y=0, 
       y_offset=-58,
       text=['AFTER program,'],
       text_font_size='13pt', 
       text_color=GRAY_4)

p.text(x=55, 
       y=0,
       y_offset=-10,
       text=['\t' * 28 + 'more children \nwere Kind of interested (30%) \n& Excited (38%)'],
       text_font_size='13pt', 
       text_color=BLUE_2)

p.text(x=55, 
       y=0,
       y_offset=-10,
       text=['\t' * 27 + 'about science.'],
       text_font_size='13pt', 
       text_color=GRAY_4)


# Elements attributes

# Modify title attributes
p.title.text_color = 'white'
p.title.background_fill_color = GRAY_2
p.title.text_font = FONT
p.title.text_font_size = '20pt'
p.title.text_font_style = 'normal'


# Modify X axis attributes
p.xaxis.bounds = (0, 100)
p.xaxis.axis_line_color = GRAY_9
p.xgrid.grid_line_color = None
p.xaxis.major_tick_line_color = GRAY_9
p.xaxis.major_label_text_color = GRAY_9
p.xaxis.major_label_text_font_size = '11pt'
p.xaxis.minor_tick_line_color = None
p.xaxis.formatter = PrintfTickFormatter(format='%0.0f %%')


# Modify Y axis attributes
p.yaxis.axis_line_color = None
p.ygrid.grid_line_color = None
p.yaxis.major_tick_line_color = None
p.yaxis.major_label_text_color = GRAY_4
p.yaxis.major_label_text_font_size = '14pt'
p.yaxis.major_label_standoff = -80

# Convert the figure to png
p.outline_line_color = None
p.background_fill_color = None
p.border_fill_color = None

show(p)